# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 4
iteration = "-7"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 5
num_batch = 2000 #25000 #11112
val_batch = 20
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

#             if save_masks and phase == 'val':
#                 outputs = sigmoid_function(outputs)
#                 # need to fix before running
#                 outputs_dir = "data/outputs/results/" + str(datetime.now().date())
#                 os.makedirs(outputs_dir, exist_ok=True)
#                 threshold = 0.05
#                 j = 0
#                 for locations in paths:
#                     img = Image.open(locations)
#                     #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     for threshold in [0.05, .25, .5, .75]:
#                         img = data_factory.showMaskOnImage(img,outputs[j], threshold)
#                         img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
#                     j += 1

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
            #torch.save(model, SAVE_LOCATION)
            #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    #torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    #torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/4
Batch 1/2000
loss: 0.2746497392654419
---------------
Batch 2/2000
loss: 0.34615376591682434
---------------
Batch 3/2000
loss: 0.3211914002895355
---------------
Batch 4/2000
loss: 0.3524310290813446
---------------
Batch 5/2000
loss: 0.38253483176231384
---------------
Batch 6/2000
loss: 0.2733961045742035
---------------
Batch 7/2000
loss: 0.4227116107940674
---------------
Batch 8/2000
loss: 0.2742534875869751
---------------
Batch 9/2000
loss: 0.39149537682533264
---------------
Batch 10/2000
loss: 0.42194581031799316
---------------
Batch 11/2000
loss: 0.31432461738586426
---------------
Batch 12/2000
loss: 0.3909665048122406
---------------
Batch 13/2000
loss: 0.28758418560028076
---------------
Batch 14/2000
loss: 0.4222446084022522
---------------
Batch 15/2000
loss: 0.33095499873161316
---------------
Batch 16/2000
loss: 0.37427863478660583
---------------
Batch 17/2000
loss: 0.35592010617256165
---------------
Batch 18/2000
loss: 0.4252721667289734
---------------


Batch 148/2000
loss: 0.4085851013660431
---------------
Batch 149/2000
loss: 0.3251588046550751
---------------
Batch 150/2000
loss: 0.28940674662590027
---------------
Batch 151/2000
loss: 0.17633891105651855
---------------
Batch 152/2000
loss: 0.3416799306869507
---------------
Batch 153/2000
loss: 0.25827550888061523
---------------
Batch 154/2000
loss: 0.39765170216560364
---------------
Batch 155/2000
loss: 0.2399846762418747
---------------
Batch 156/2000
loss: 0.280135840177536
---------------
Batch 157/2000
loss: 0.303763747215271
---------------
Batch 158/2000
loss: 0.2656593918800354
---------------
Batch 159/2000
loss: 0.3117767572402954
---------------
Batch 160/2000
loss: 0.25162985920906067
---------------
Batch 161/2000
loss: 0.36431175470352173
---------------
Batch 162/2000
loss: 0.38178741931915283
---------------
Batch 163/2000
loss: 0.31972837448120117
---------------
Batch 164/2000
loss: 0.320450097322464
---------------
Batch 165/2000
loss: 0.30829188227653503
--

loss: 0.328958123922348
---------------
Batch 294/2000
loss: 0.36157888174057007
---------------
Batch 295/2000
loss: 0.33989202976226807
---------------
Batch 296/2000
loss: 0.2728688716888428
---------------
Batch 297/2000
loss: 0.29304996132850647
---------------
Batch 298/2000
loss: 0.3768097758293152
---------------
Batch 299/2000
loss: 0.3145098388195038
---------------
Batch 300/2000
loss: 0.4162026345729828
---------------
Batch 301/2000
loss: 0.3827494978904724
---------------
Batch 302/2000
loss: 0.3444819748401642
---------------
Batch 303/2000
loss: 0.31423163414001465
---------------
Batch 304/2000
loss: 0.3011324107646942
---------------
Batch 305/2000
loss: 0.3182829022407532
---------------
Batch 306/2000
loss: 0.29051342606544495
---------------
Batch 307/2000
loss: 0.32975444197654724
---------------
Batch 308/2000
loss: 0.31133219599723816
---------------
Batch 309/2000
loss: 0.4142608940601349
---------------
Batch 310/2000
loss: 0.3553627133369446
---------------
B

---------------
Batch 439/2000
loss: 0.3340211510658264
---------------
Batch 440/2000
loss: 0.22961321473121643
---------------
Batch 441/2000
loss: 0.27839523553848267
---------------
Batch 442/2000
loss: 0.24216409027576447
---------------
Batch 443/2000
loss: 0.31358006596565247
---------------
Batch 444/2000
loss: 0.2593192756175995
---------------
Batch 445/2000
loss: 0.2383972555398941
---------------
Batch 446/2000
loss: 0.3057937026023865
---------------
Batch 447/2000
loss: 0.4267185926437378
---------------
Batch 448/2000
loss: 0.3141721487045288
---------------
Batch 449/2000
loss: 0.35258471965789795
---------------
Batch 450/2000
loss: 0.48012199997901917
---------------
Batch 451/2000
loss: 0.34570616483688354
---------------
Batch 452/2000
loss: 0.2873300313949585
---------------
Batch 453/2000
loss: 0.34362295269966125
---------------
Batch 454/2000
loss: 0.3812195956707001
---------------
Batch 455/2000
loss: 0.3457348942756653
---------------
Batch 456/2000
loss: 0.2

Batch 584/2000
loss: 0.3378162980079651
---------------
Batch 585/2000
loss: 0.3565264642238617
---------------
Batch 586/2000
loss: 0.35126352310180664
---------------
Batch 587/2000
loss: 0.2982597053050995
---------------
Batch 588/2000
loss: 0.2859518229961395
---------------
Batch 589/2000
loss: 0.2691669464111328
---------------
Batch 590/2000
loss: 0.2568879723548889
---------------
Batch 591/2000
loss: 0.328264981508255
---------------
Batch 592/2000
loss: 0.2874261736869812
---------------
Batch 593/2000
loss: 0.30678877234458923
---------------
Batch 594/2000
loss: 0.29254063963890076
---------------
Batch 595/2000
loss: 0.25085684657096863
---------------
Batch 596/2000
loss: 0.25670796632766724
---------------
Batch 597/2000
loss: 0.26585710048675537
---------------
Batch 598/2000
loss: 0.4138413667678833
---------------
Batch 599/2000
loss: 0.36409759521484375
---------------
Batch 600/2000
loss: 0.336890310049057
---------------
Batch 601/2000
loss: 0.2762098014354706
---

...

Batch 1449/2000
loss: 0.2011907547712326
---------------
Batch 1450/2000
loss: 0.34454137086868286
---------------
Batch 1451/2000
loss: 0.45490390062332153
---------------
Batch 1452/2000
loss: 0.35995450615882874
---------------
Batch 1453/2000
loss: 0.3136482536792755
---------------
Batch 1454/2000
loss: 0.2917621433734894
---------------
Batch 1455/2000
loss: 0.28803375363349915
---------------
Batch 1456/2000
loss: 0.22812803089618683
---------------
Batch 1457/2000
loss: 0.33676373958587646
---------------
Batch 1458/2000
loss: 0.2977716624736786
---------------
Batch 1459/2000
loss: 0.3156707286834717
---------------
Batch 1460/2000
loss: 0.4412623941898346
---------------
Batch 1461/2000
loss: 0.2929602265357971
---------------
Batch 1462/2000
loss: 0.3532366454601288
---------------
Batch 1463/2000
loss: 0.27838853001594543
---------------
Batch 1464/2000
loss: 0.395696759223938
---------------
Batch 1465/2000
loss: 0.24459631741046906
---------------
Batch 1466/2000
loss: 0.

Batch 1592/2000
loss: 0.25034984946250916
---------------
Batch 1593/2000
loss: 0.41350796818733215
---------------
Batch 1594/2000
loss: 0.280830442905426
---------------
Batch 1595/2000
loss: 0.2736564576625824
---------------
Batch 1596/2000
loss: 0.36383599042892456
---------------
Batch 1597/2000
loss: 0.26478904485702515
---------------
Batch 1598/2000
loss: 0.36982646584510803
---------------
Batch 1599/2000
loss: 0.3016270399093628
---------------
Batch 1600/2000
loss: 0.3655811548233032
---------------
Batch 1601/2000
loss: 0.28372228145599365
---------------
Batch 1602/2000
loss: 0.3231185972690582
---------------
Batch 1603/2000
loss: 0.3680911064147949
---------------
Batch 1604/2000
loss: 0.3160948157310486
---------------
Batch 1605/2000
loss: 0.29803767800331116
---------------
Batch 1606/2000
loss: 0.37880125641822815
---------------
Batch 1607/2000
loss: 0.3039127588272095
---------------
Batch 1608/2000
loss: 0.257473349571228
---------------
Batch 1609/2000
loss: 0.3

Batch 1735/2000
loss: 0.3812020421028137
---------------
Batch 1736/2000
loss: 0.33824634552001953
---------------
Batch 1737/2000
loss: 0.3124932050704956
---------------
Batch 1738/2000
loss: 0.39472100138664246
---------------
Batch 1739/2000
loss: 0.33229291439056396
---------------
Batch 1740/2000
loss: 0.34009426832199097
---------------
Batch 1741/2000
loss: 0.262824147939682
---------------
Batch 1742/2000
loss: 0.23842394351959229
---------------
Batch 1743/2000
loss: 0.45921897888183594
---------------
Batch 1744/2000
loss: 0.2436532825231552
---------------
Batch 1745/2000
loss: 0.33320045471191406
---------------
Batch 1746/2000
loss: 0.338196337223053
---------------
Batch 1747/2000
loss: 0.2822742462158203
---------------
Batch 1748/2000
loss: 0.34188395738601685
---------------
Batch 1749/2000
loss: 0.3327106833457947
---------------
Batch 1750/2000
loss: 0.2570689022541046
---------------
Batch 1751/2000
loss: 0.27743348479270935
---------------
Batch 1752/2000
loss: 0.

---------------
Batch 1878/2000
loss: 0.304170697927475
---------------
Batch 1879/2000
loss: 0.2951478958129883
---------------
Batch 1880/2000
loss: 0.29817694425582886
---------------
Batch 1881/2000
loss: 0.33665138483047485
---------------
Batch 1882/2000
loss: 0.32855135202407837
---------------
Batch 1883/2000
loss: 0.3813861310482025
---------------
Batch 1884/2000
loss: 0.401117742061615
---------------
Batch 1885/2000
loss: 0.4266422390937805
---------------
Batch 1886/2000
loss: 0.23200199007987976
---------------
Batch 1887/2000
loss: 0.3401409089565277
---------------
Batch 1888/2000
loss: 0.3181605637073517
---------------
Batch 1889/2000
loss: 0.3370681703090668
---------------
Batch 1890/2000
loss: 0.29966747760772705
---------------
Batch 1891/2000
loss: 0.23166882991790771
---------------
Batch 1892/2000
loss: 0.3654577136039734
---------------
Batch 1893/2000
loss: 0.27494674921035767
---------------
Batch 1894/2000
loss: 0.32133883237838745
---------------
Batch 189

---------------
Training complete in 653.0m 31.0s


# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [8]:
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(